In [27]:
#import libraries
from sys import argv
import time
import numpy as np
import math
import random
import copy

In [28]:
#Set the container of pieces
BLACK = 1
WHITE = -1
EMPTY = 0

color_name = {BLACK: 'Black', WHITE: 'White'}
opponent = {BLACK: WHITE, WHITE: BLACK}

WHITE_PIECE = 'O'
BLACK_PIECE = 'X'

In [29]:
#Utility function
def is_in_bounds(x, y, size):
    return 0 <= x < size and 0 <= y < size

In [30]:
class Board:
    """
    Creates the board for the game.
    """

    def __init__(self, size):
        self.size = 15
        self.black_stones = 0
        self.white_stones = 0
        self.board = [[EMPTY for _ in range(15)]
                      for _ in range(15)]
        self.init_starting_position()

    def init_starting_position(self):
        self.board = [[EMPTY for _ in range(15)]
                      for _ in range(15)]

        self.black_stones = 0
        self.white_stones = 0
        
    def place_stone_at(self, color, x, y):
        self.board[y][x] = color
        if color == WHITE:
            self.white_stones += 1
        elif color == BLACK:
            self.black_stones += 1

    def is_full(self):
        return self.black_stones + self.white_stones == (15 ** 2)

    def __str__(self):
        Dic = {14 : "O", 13 : "N", 12 :"M", 11 : "L", 10 : "K", 9 : "J", 8 : "I", 
               7 : "H", 6  : "G", 5  : "F", 4: "E", 3 : "D", 2 : "C", 1 : "B", 0 :"A"}

        result = '   '
            
        x = [ ' ' + str(i) + ' '  for i in range(0,15,1)]

        for y in range(15):
            if y < 10:
                result += ' ' + str(y) + ' '
            else:
                result += str(y) + ' '
        result += '\n   '
        for x in range(15):
            result += '___'
        result += '_'    
        for y in range(0, 15, 1):
            result += '\n' + Dic[y] + '| '
            
            for x in range(15):
                if self.board[y][x] == WHITE:
                    result += ' ' + WHITE_PIECE + ' '
                elif self.board[y][x] == BLACK:
                    result += ' ' +BLACK_PIECE + ' '
                elif self.board[y][x] == EMPTY:
                    result += ' - '
                else:
                    result += '? '

        return result

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        if self.black_stones != other.black_stones:
            return False
        if self.white_stones != other.white_stones: 
            return False
        return self.board == other.board
    
    def __hash__(self):
        hash = 5138

        for y in range(15):
            for x in range(15):
                hash += self.board[y][x]
                hash += (hash << 10)
                hash ^= (hash >> 6)

        hash += (hash << 3)
        hash ^= (hash >> 11)
        hash += (hash << 15)
        return hash
    
    def __getitem__(self, key):
        return self.board[key]
    
    def get_size(self):
        return self.size
    
    """
    Below are functions that return an array from the given position (y,x) with length 5
    and a list of the coordinates of that line.
    """
    
    def column_line(self,y, x, length=5):
        line = np.empty(length, dtype='int8')
        for i in range(length):
            line[i] = np.array(self.board)[y+i,x]
        return line, [(y+i,x) for i in range(length)]
    
    def row_line(self, y, x, length=5):
        line = np.empty(length, dtype='int8')
        for i in range(length):
            line[i] = np.array(self.board)[y,x+i]
        return line, [(y,x+i) for i in range(length)]

    def diagonal_upleft_lowright_line(self, y, x, length=5):
        line = np.empty(length, dtype='int8')
        for i in range(length):
            line[i] = np.array(self.board)[y+i,x+i]
        return line, [(y+i,x+i) for i in range(length)]

    def diagonal_lowleft_upright_line(self, y, x, length=5):
        line = np.empty(length, dtype='int8')
        if y < length - 1:
            raise IndexError
        for i in range(length):
            line[i] = np.array(self.board)[y-i,x+i]
        return line, [(y-i,x+i) for i in range(length)]

In [31]:
class Agent:
    """
    Defines the interface of the Agents (Human, Random, and MonteCarlo).
    """
    def __init__(self, fiveinaline, color):
        raise NotImplementedError

    def get_action(self, game_state, legal_moves=None):
        raise NotImplementedError

    def observe_win(self, state, winner):
        raise NotImplementedError

    def reset(self):
        raise NotImplementedError

In [32]:
class RandomAgent(Agent):
    """Chooses random legal moves."""

    def __init__(self, fiveinaline, color, **kwargs):
        self.fiveinaline = fiveinaline
        self.color = color

    def get_action(self, state, legal_moves):
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def reset(self):
        pass

    def observe_win(self, winner):
        pass

    def legal_moves(self, game_state):

        board_size = game_state[0].size
        moves = []

        for y in range(board_size):
            for x in range(board_size):
                if self.is_valid_move(game_state, x, y):
                    moves.append((x, y))
        return moves
    
    @staticmethod
    def is_valid_move(game_state, x, y):
        board, color = game_state
        piece = board.board[y][x]
        if piece != EMPTY:
            return False

        return True

In [33]:
class OffenseDefense(object):
    """
    This contains the functions that an AI can use to make defense and offense in a Five-in-a-line game.
    These apply the domain knowledge from pappers about Five-in-a-line.
    """
    
    def  secondmove(self,picked):
        x,y=picked[0], picked[1]

        if y <= int(np.round(15/2)):
            dy = 1
        else: dy = -1

        if x <=int(np.round(15/2)):
            dx = 1
        else: dx = -1

        return (y+dy,x+dx)
    
    def get_line_position(self, game_state, length=5):
        return [lambda x,y: game_state.column_line(x,y,length=length), lambda x,y: game_state.row_line(x,y, length=length),
                lambda x,y: game_state.diagonal_upleft_lowright_line(x,y, length=length),
                lambda x,y: game_state.diagonal_lowleft_upright_line(x,y, length=length)]
    
    def block_three(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                        line, positions = f(i,j)
                    except IndexError:
                        continue

                    if len(np.where(line == EMPTY)[0]) == 2 and len(np.where(line == -color)[0]) == 3:
                        indices_opponent = np.where(line == -color)[0]
                        if not (indices_opponent[1] == indices_opponent[0] + 1 and \
                                indices_opponent[2] == indices_opponent[1] + 1):
                                    continue
                        if 0 not in indices_opponent:
                            move = (positions[indices_opponent[0] - 1][1], positions[indices_opponent[0] - 1][0])
                            return move
                        else:
                            move = (positions[3][1], positions[3][0])
                            return move
        return False
    
    def block_four(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                        line, positions = f(i,j)
                    except IndexError:
                        continue

                    if len(np.where(line == EMPTY)[0]) == 1 and len(np.where(line == -color)[0]) == 4:
                        index_of_empty = np.where(line == EMPTY)[0][0]
                        move = (positions[index_of_empty][1],positions[index_of_empty][0])
                        return move
        return False
    
    def block_double_two(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                        line, positions = f(i,j)
                    except IndexError:
                        continue

                    if ( line == (EMPTY, -color, -color, EMPTY, EMPTY) ).all():
                        position = positions[3]
                        move = (position[1], position[0])
                        return move

                    elif ( line == (EMPTY, EMPTY, -color, -color, EMPTY) ).all():
                        position = positions[1]
                        move = (position[1], position[0])
                        return move
        return False
    
    def block_double_three(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                        line, positions = f(i,j)
                    except IndexError:
                        continue

                    if ( line == (EMPTY, -color, -color, -color, EMPTY) ).all():
                        position = positions[0]
                        move = (position[1], position[0])
                        return move
        return False
    
    def extend_two(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                        line, positions = f(i,j)
                    except IndexError:
                        continue

                    if len(np.where(line == EMPTY)[0]) == 3 and len(np.where(line == color)[0]) == 2:
                        indices_empty = np.where(line == EMPTY)[0]
                        position = positions[indices_empty[np.random.randint(3)]]
                        move = (position[1], position[0])
                        return move
        return False
    
    def extend_three(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                        line, positions = f(i,j)
                    except IndexError:
                        continue
                        
                    if len(np.where(line == EMPTY)[0]) == 2 and len(np.where(line == color)[0]) == 3:
                        indices_empty = np.where(line == EMPTY)[0]
                        if 0 not in indices_empty:
                            position = positions[indices_empty[0]]
                            move = (position[1], position[0])
                            return move
                        else:
                            position = positions[indices_empty[1]]
                            move = (position[1], position[0])
                            return move
        return False
    
    def possible_win(self, game_state,color):
        for i in range(game_state.size):
            for j in range(game_state.size):
                for f in self.get_line_position(game_state):
                    try:
                         line, positions = f(i,j)
                    except IndexError:
                         continue
                            
                    if EMPTY in line and line.sum() == color * 4:
                        for pos in positions:
                            if game_state.board[pos[0]][pos[1]] == EMPTY:
                                return (pos[1], pos[0])

In [34]:
class MonteCarloAgent(Agent):

    def __init__(self, fiveinaline, color, **kwargs):
        self.color = color
        self.fiveinaline = fiveinaline
        self.sim_time = kwargs.get('sim_time', 5)

        self.state_node = {} #contains game state, node, # of wins and plays

    def reset(self):
        pass

    def observe_win(self, winner):
        pass


    def get_action(self, game_state, legal_moves):
        """
        For each game state and legal moves, the MCTS agent decides a move. 
        It prioritizes in this  order: Defense - Offense - Explore/Exploit using MCTS.
        This is used by the FiveInALine game object.
        """
        
        defense = OffenseDefense()
        
        board = game_state[0]
        if board.black_stones + board.white_stones == 0:
            x = int(np.round(board.size/2))
            return (x-1,x-1)
        
        if board.black_stones + board.white_stones == 1:
            val=1 #want to know where the opponent puts its move
            picked = [(index, row.index(val)) for index, row in enumerate(board.board) if val in row][0]
            return defense.secondmove(picked)
        
        if defense.possible_win(board,self.color) != None: 
            return defense.possible_win(board,self.color)
        
        if defense.block_four(board,self.color) != False: 
            return defense.block_four(board,self.color)
        
        if defense.block_three(board,self.color) != False: 
            return defense.block_three(board,self.color)
               
        if defense.block_double_two(board,self.color) != False: 
            return defense.block_double_two(board,self.color)
        
        if defense.block_double_three(board,self.color) != False: 
            return defense.block_double_three(board,self.color)
        
        if defense.extend_three(board,self.color) != False: 
            return defense.extend_three(board,self.color)
        
        if defense.extend_two(board,self.color) != False: 
            return defense.extend_two(board,self.color)

        if not legal_moves:
            return None
        game_state = copy.deepcopy(game_state) #to avoid mutation
        move = self.monte_carlo_search(game_state)
        return move

    def monte_carlo_search(self, game_state):
        """
        For a given state of the game, this will return the best action based from
        Monte Carlo Tree Search algorithm with an Upper Confidence Bound.
        """
        results = {}  # map position to wins/plays
        root = None

        if game_state in self.state_node:
            root = self.state_node[game_state]

        else:
            amnt_children = len(self.legal_moves(game_state))
            if self.fiveinaline.winner(game_state) is False and amnt_children == 0:
                amnt_children = 1

            root = Node(game_state, None, amnt_children)
            
        root.parent = None
        
        sim_count = 0
        now = time.time()
        
        while time.time() - now < self.sim_time and root.moves_unfinished > 0:
            picked_node = self.tree_policy(root)
            result = self.simulate(picked_node.game_state)

            self.back_prop(picked_node, result)
            sim_count += 1

        for child in root.children:
            wins, plays = child.get_wins_plays()
            position = child.move
            results[position] = (wins, plays)
  
        return self.best_action(root)

    @staticmethod
    def best_action(node):
        """
        For a given game state node, return the best action.
        MCTS picks the most visited node. To break ties, the number of wins is used.
        """
        most_plays = -float('inf')
        best_wins = -float('inf')
        best_actions = []
        
        for child in node.children:
            wins, plays = child.get_wins_plays()
            if plays > most_plays:
                most_plays = plays
                best_actions = [child.move]
                best_wins = wins


            elif plays == most_plays:
                if wins > best_wins:
                    best_wins = wins
                    best_actions = [child.move]

                elif wins == best_wins:
                    best_actions.append(child.move)
                    
        return random.choice(best_actions)

    @staticmethod
    def back_prop(node, delta):
        """
        Propagate the information up the tree to the root.
        """
        while node.parent is not None:
            
            node.plays += 1
            node.wins += delta
            node = node.parent
            
        node.plays += 1
        node.wins += delta

    def tree_policy(self, root):
        """
        Executes the Selection - Expansion steps of MCTS algorithm.
        Given a root node, determine which child to visit
        using Upper Confidence Bound.
        """
        cur_node = root
        
        while True and root.moves_unfinished > 0:
            legal_moves = self.legal_moves(cur_node.game_state)
            
            if not legal_moves:
                if self.fiveinaline.winner(cur_node.game_state) is not False:
                    cur_node.propagate_completion()
                    return cur_node

            elif len(cur_node.children) < len(legal_moves):#expansion
                unexpanded = [
                    move for move in legal_moves
                    if move not in cur_node.moves_expanded
                ]
                
                assert len(unexpanded) > 0
                move = random.choice(unexpanded)
                state = self.fiveinaline.next_state(cur_node.game_state, move)
                child = Node(state, move, len(legal_moves))
                cur_node.add_child(child)
                self.state_node[state] = child
                return child

            else:
                cur_node = self.best_child(cur_node)
        return cur_node

    def best_child(self, node):
        enemy_turn = (node.game_state[1] != self.color)
        
        C = 1  #controls 'exploration'
        values = {}
        
        for child in node.children:
            wins, plays = child.get_wins_plays()
            if enemy_turn:
                wins = plays - wins
            _, parent_plays = node.get_wins_plays()
            
            assert parent_plays > 0
            values[child] = (wins / plays) \
                + C * math.sqrt(2 * math.log(parent_plays) / plays)
            
        best_choice = max(values, key=values.get)
        
        return best_choice

    def simulate(self, game_state):
        """
        From the given game state, simulate
        a random game to completion, and returns a value 1 for a win or 0 for a loss.
        """
        WIN_PRIZE = 1
        LOSS_PRIZE = 0
        state = copy.deepcopy(game_state)
        while True:
            winner = self.fiveinaline.winner(state)
            if winner is not False:
                if winner == self.color:
                    return WIN_PRIZE
                elif winner == opponent[self.color]:
                    return LOSS_PRIZE
                else:
                    raise ValueError

            moves = self.legal_moves(state)

            picked = random.choice(moves)
            state = self.fiveinaline.apply_move(state, picked)   
            
            
    def legal_moves(self, game_state):
        board_size = game_state[0].size
        moves = [] 

        for y in range(board_size):
            for x in range(board_size):
                if self.is_valid_move(game_state, x, y):
                    moves.append((x, y))

        return moves

    @staticmethod
    def is_valid_move(game_state, x, y):
        board, color = game_state
        piece = board.board[y][x]
        if piece != EMPTY:
            return False

        enemy = opponent[color]

        for dy in range(-1, 2):
            for dx in range(-1, 2):
                if dy == 0 and dx == 0:
                    continue

                #restrict search space: should have at least 1 opponent piece
                distance = 1
                yp = (distance * dy) + y
                xp = (distance * dx) + x

                while is_in_bounds(xp, yp, board.size) and board.board[yp][xp] == enemy:
                    distance += 1
                    yp = (distance * dy) + y
                    xp = (distance * dx) + x
                if distance > 1 and is_in_bounds(xp, yp, board.size) or (yp in (0,board.size-1)):
                    return True
        return False

class Node:
    def __init__(self, game_state, move, amount_children):
        self.game_state = game_state
        self.plays = 0
        self.wins = 0
        self.children = []
        self.parent = None
        self.moves_expanded = set() 
        self.moves_unfinished = amount_children 

        self.move = move

    def propagate_completion(self):
        """
        If all children of this move have each been expanded to
        completion, tell the parent that it has one fewer children
        left to expand.
        """

        if self.parent is None:
            return
        
        if self.moves_unfinished > 0:
            self.moves_unfinished -= 1

        self.parent.propagate_completion()

    def add_child(self, node):
        self.children.append(node)
        self.moves_expanded.add(node.move)
        node.parent = self

    def has_children(self):
        return len(self.children) > 0

    def get_wins_plays(self):
        return self.wins, self.plays

    def __hash__(self):
        return hash(self.game_state)

    def __repr__(self):
        return 'move: {} wins: {} plays: {}'.format(self.move, self.wins, self.plays)

    def __eq__(self, other):
        if not isinstance(other, Node):
            return False
        return self.game_state == other.game_state


In [35]:
class HumanAgent(Agent):
    """Agent played by a human."""

    def __init__(self, fiveinaline, color, **kwargs):
        self.fiveinaline = fiveinaline
        self.color = color

    def reset(self):
        pass

    def observe_win(self, winner):
        pass

    def get_action(self, game_state, legal):
        
        legal = self.legal_moves(game_state)
        
        Dic = {"O" : 14, "N" : 13, "M" : 12, "L" : 11, "K" : 10, "J" : 9, "I" : 8,
               "H" : 7, "G"  : 6, "F"  : 5,"E" : 4, "D" : 3, "C" : 2, "B" : 1, "A" : 0}
        
        if not legal:
            return None
        choice = None
        while True:
            raw_choice = input('Enter a move x,y: ')
            if raw_choice == 'pass':
                return None
            elif raw_choice == 'exit' or raw_choice == 'quit':
                quit()
                
            elif len(raw_choice) != 3 and len(raw_choice) != 4:
                print('Input must be 3 or 4 characters long, formatted x,y')                    
                continue

            if raw_choice[1] != ',':
                print('Comma separator not found.')
                continue
            if not raw_choice[0].isalpha() or not raw_choice[2].isdigit():
                print('Couldn\'t determine x,y from your input.')
                continue

            if raw_choice[0] == "P" or raw_choice[0] ==  "Q" or raw_choice[0] == "R" or raw_choice[0] == "S" or raw_choice[0] == "T" or raw_choice[0] == "U" or raw_choice[0] == "V" or raw_choice[0] == "W" or raw_choice[0] == "X" or raw_choice[0] == "Y" or raw_choice[0] == "Z" or raw_choice[0] == "p" or raw_choice[0] == "q" or raw_choice[0] == "r" or raw_choice[0] == "s" or raw_choice[0] == "t" or raw_choice[0] == "u" or raw_choice[0] == "v" or raw_choice[0] == "w" or raw_choice[0] == "x" or raw_choice[0] == "y" or raw_choice[0] == "z":
                print('Only accept letters from A to O.')
                continue
                 
            raw_choices_upper = raw_choice[0].upper()
            raw_choices = Dic[raw_choices_upper]
               
            choice = (int(raw_choice[2]), int(raw_choices))
            
            if len(raw_choice) == 4:
                choicerow = raw_choice[2]+ raw_choice[3]
                choice = (int(choicerow), int(raw_choices))
                
            if choice not in legal:
                print('Not a legal move. Try again.')
                continue
                
            else:
                break

        return choice
    
    def legal_moves(self, game_state):

        board_size = game_state[0].size
        moves = [] 

        for y in range(board_size):
            for x in range(board_size):
                if self.is_valid_move(game_state, x, y):
                    moves.append((x, y))
        return moves
    
    @staticmethod
    def is_valid_move(game_state, x, y):
        board, color = game_state
        piece = board.board[y][x]
        if piece != EMPTY: 
            return False

        return True

In [36]:
class FiveInALine:
    """This class gives the rules for the Five-in-a-line game."""

    def __init__(self, **kwargs):
        self.size = kwargs.get('size', 15)
        self.board = Board(self.size)

        WhiteAgent = kwargs.get('WhiteAgent', RandomAgent)
        BlackAgent = kwargs.get('BlackAgent', RandomAgent)
        self.white_agent = WhiteAgent(self, WHITE, **kwargs)
        self.black_agent = BlackAgent(self, BLACK, **kwargs)

        self.reset()

    def reset(self):
        self.board.init_starting_position()
        self.game_state = (self.board, BLACK)

        self.white_agent.reset()
        self.black_agent.reset()

    def play_game(self):
        
        state = self.get_state()
        self.print_board(state)
        print("===================================================")
        print("")       
        
        while self.winner(state) is False:
            color = state[1]
            
            picked = self.agent_pick_move(state)
            state = self.next_state(state, picked)
            self.print_board(state)
            
            print("")
            
            Dic = {14 : "O", 13 : "N", 12 :"M", 11 : "L", 10 : "K", 9 : "J", 8 : "I", 7 : "H",
                   6  : "G", 5  : "F", 4: "E", 3 : "D", 2 : "C", 1 : "B", 0 :"A"}
        
            picked_ = (Dic[picked[1]], int(picked[0]))
            
            print('{} plays at {}'.format(color_name[color], str(picked_)))
            
            print("===================================================")
            print("")
            
        winner = color
        
        self.reset()
        return winner

    @staticmethod

    def print_board(state):
        board = state[0]
        print(board)
        
    def agent_pick_move(self, state):
        color = state[1]
        picked = None
        if color == WHITE:
            legal_moves = self.white_agent.legal_moves(state)
            picked = self.white_agent.get_action(state, legal_moves)
        elif color == BLACK:
            legal_moves = self.black_agent.legal_moves(state)
            picked = self.black_agent.get_action(state, legal_moves)
        else:
            raise ValueError

        if picked is None:
            return None

        return picked

    def next_state(self, game_state, move):
        return self.apply_move(copy.deepcopy(game_state), move)

    @staticmethod
    def apply_move(game_state, move):
        if not move:
            game_state = (game_state[0], opponent[game_state[1]])
            return game_state

        x, y = move
        board,color = game_state
        board.place_stone_at(color, x, y)

        enemy_color = BLACK
        if color == BLACK:
            enemy_color = WHITE

        game_state = (game_state[0], opponent[game_state[1]])
        return game_state
    
    def winner(self, game_state):
        color = game_state[1]
        board = game_state[0].board
        
        #horizontal
        for X in range (15):
                for Y in range (11):
                    countx = 0
                    county = 0
                    for Y in range (Y, Y+5, 1):
                        if board[X][Y]== -1:
                            countx += 1
                        if board[X][Y]== 1:
                            county += 1
                    if Y == 15:
                        break
                    if countx == 5 or county == 5:
                        return color

        #vertical                  
        for Y in range (15):
                for X in range (11):
                    countx = 0
                    county = 0
                    for X in range (X, X+5, 1):
                        if board[X][Y]== -1:
                            countx += 1
                        if board[X][Y]== 1:
                            county += 1
                    if X == 15:
                        break
                    if countx == 5 or county == 5:
                        return color

        #diagonal upper right to lower left
        for X in range (11): 
            for Y in range (11):
                countx = 0
                county = 0
                N = X
                M = Y
                for Z in range (5):
                    if board[N][M]== -1:
                        countx += 1
                    if board[N][M]== 1:
                        county += 1
                    N += 1
                    M += 1
                if countx == 5 or county == 5:
                    return color

        #diagonal upper left to lower right
        for X in range (11):  
            for Y in range (14, 3, -1):
                countx = 0
                county = 0
                N = X
                M = Y
                for Z in range (5):
                    if board[N][M]== -1:
                        countx += 1
                    if board[N][M]== 1:
                        county += 1
                    N += 1
                    M -= 1
                if countx == 5 or county == 5:
                    return color
                
        return False

    def get_state(self):
        return self.game_state

In [37]:
def prop_parse(props):
    result = {}

    for arg in props:
        split = arg.split('=')
        if len(split) != 2:
            continue
        property = split[0]
        value = split[1]

        if value == 'True':
            value = True
        elif value == 'False':
            value = False
        else:
            try:
                value = float(value)
                if value.is_integer():
                    value = int(value)
            except ValueError:
                pass

        result[property] = value

    return result

In [38]:
prop_names = {
        'monte_carlo1': MonteCarloAgent,
        'monte_carlo2': MonteCarloAgent,
        'random': RandomAgent,
        'human': HumanAgent,
        }

def main(**kwargs):

    input_args = prop_parse(argv)
    input_args.update(kwargs)

    if len(argv) <= 1 and len(kwargs) <= 1:
        print('necessary inputs:')
        print('BlackAgent=, WhiteAgent=,')
        print('choices: monte_carlo, random, human')
        quit()

    for k, v in input_args.items():
        if v in prop_names:
            input_args[k] = prop_names[v]

    if any(val == MonteCarloAgent for val in input_args.values()) \
            and not input_args.get('sim_time', False):
        print('sim_time field required for monte_carlo agent.')
        print('quitting.')
        quit()

    print("")
    print("              THE GAME BEGINS!")
    print('Black as {}, White as {}.'.format(
        input_args['BlackAgent'].__name__, input_args['WhiteAgent'].__name__)
        )
    print("")
    print("===================================================")
    print("")
    
    fiveinaline = FiveInALine(**input_args)
    start = time.time()

    winner  = fiveinaline.play_game()
    
    print("===================== GAME OVER ====================")
    print("")
    print('                     {} wins!'.format(color_name[winner]))
    print('                 time: {0:.2f} minutes'.format((time.time() - start) / 60))
    print("")
    print("===================================================")

In [41]:
main(BlackAgent='human', WhiteAgent='monte_carlo2', sim_time=8)


              THE GAME BEGINS!
Black as HumanAgent, White as MonteCarloAgent.


    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 
   ______________________________________________
A|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
B|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
C|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
D|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
E|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
F|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
G|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
H|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
I|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
J|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
K|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
L|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
M|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
N|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
O|  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 

Enter a move x,y: m,7
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 
   __________